# Customization

In [10]:
USERNAME = 'joloujo'
TIMEZONE = 'US/Eastern'

## Set up pyLast

In [2]:
from dotenv import load_dotenv
import os
import pylast

# Load environment variables from .env file
load_dotenv()
API_KEY = os.getenv('PYLAST_API_KEY')
API_SECRET = os.getenv('PYLAST_API_SECRET')

# Make sure environment variables were successfully loaded
if API_KEY == None or API_SECRET == None:
    if API_KEY == None:
        print('API Key not found')
    if API_SECRET == None:
        print('API Secret not found')
    print('Exiting')
else:
    network = pylast.LastFMNetwork(API_KEY, API_SECRET)
    network.enable_rate_limit()
    print('Successfully set up pyLast')


Successfully set up pyLast


## Load scrobbles

In [15]:
from datetime import datetime
from pathlib import Path
import pickle
from pylast import PlayedTrack
import pytz
import time

tracks: list[PlayedTrack] = []
tracks_path = Path("..") / "data" / "tracks.pkl"

if os.path.exists(tracks_path):
    with open(tracks_path, 'rb') as tracks_file:
        tracks = pickle.load(tracks_file)
    print('Loaded tracks from file')
else:
    print('Tracks file not found, fetching from last.fm')
    user = network.get_user(USERNAME)

    time_to: int = round(time.time())

    while True:
        new_tracks = list(user.get_recent_tracks(limit=500, time_to=time_to))

        if len(new_tracks) == 0:
            print('Done fetching tracks')
            break

        print(f'fetched tracks {len(tracks) + 1} to {len(tracks) + len(new_tracks)}')

        last_timestamp = new_tracks[-1].timestamp
        tracks = tracks + new_tracks

        if last_timestamp == None: 
            print('Song with no timestamp encountered')
            break

        time_to = int(last_timestamp)

    with open(tracks_path, 'wb') as tracks_file:
        pickle.dump(tracks, tracks_file)
    print('Saved tracks to file')

if tracks[0].timestamp:
    timestamp = float(tracks[0].timestamp)
    timezone = pytz.timezone(TIMEZONE)
    date = datetime.fromtimestamp(timestamp, timezone)
    date_string = date.strftime("%m/%d/%Y %H:%M:%S %Z")
    print(f'Most recent track is from {date_string}')
else:
    print('Most recent track has no timestamp')



Loaded tracks from file
Most recent track is from 11/04/2025 16:00:54 EST
